# Text classification with hierarchical LSTM

TODO: intro

In [ ]:
!pip install thinc spacy

First, let's use Thinc's `prefer_gpu` helper to make sure we're performing operations **on GPU if available**. The function should be called right after importing Thinc, and it returns a boolean indicating whether the GPU has been activated.

In [ ]:
from thinc.api import prefer_gpu
is_gpu = prefer_gpu()
print("GPU:", is_gpu)

---

## Defining the model

Each batch of inputs will be received as a list of spaCy [`Doc`](https://spacy.io/api/doc) objects, and the model will need to output a vector of scores for each entry in the list. Let's start planning this out, using type-annotations to keep track of the inputs and outputs:

```
TextClassifier(...) -> Model[List[Doc], Array2d]
```

Our model will need some sort of embedding function, and some function that re-encodes the vectors based on context (such as a BiLSTM, CNN, etc). For the encoding layer, it’s usually best to work on padded batches, so that component should take a `Padded` object as input, and return a `Padded` object as output. So let’s write our function expecting the embed layer to return that type, and we’ll expect our reduction layer to transform from `Padded` to `Array2d`, which is what the prediction layer will expect.

In [ ]:
from typing import List
from thinc.api import Model, Padded, chain
from thinc.types import Array2d
from spacy.tokens import Doc

def TextClassifier(
    embed: Model[List[Doc], Padded],
    encode: Model[Padded, Padded],
    reduce: Model[Padded, Array2d],
    predict: Model[Array2d, Array2d]
) -> Model[List[Doc], Array2d]:
    return chain(embed, encode, reduce, predict)

### The `WordEmbed` layer

Now that we know all our input and output types, we can design functions to build the various components. There could be any number of combinations, but here are some reasonable defaults. The `with_array` wrapper transforms to an `Array2d`, and then reverses the transformation on the way back out. For the `Padded` object, this is just a reshape.

TODO: describe `docs2arrays`

In [ ]:
from typing import Tuple, Callable, Sequence
from thinc.api import Model
from spacy.tokens import Doc 

def docs2arrays(token_attrs: List[int]):
    def forward(model: Model[Sequence[Doc], List[Array2d]], Xs: Sequence[Doc], is_train: bool):
        token_attrs = model.get_attr("token_attrs")
        return [doc.to_array(token_attrs) for doc in Xs], lambda dX: []
    
    return Model("docs2arrays", forward, attrs={"token_attrs": token_attrs})

TODO: ...

In [ ]:
def docs2strings(token_attr: str = "text"):
    def forward(model: Model[Sequence[Doc], List[List[str]]], Xs: Sequence[Doc], is_train: bool):
        token_attr = model.get_attr("token_attr")
        return [[getattr(token, token_attr) for token in doc] for doc in Xs], lambda dX: []
    
    return Model("docs2strings", forward, attrs={"token_attr": token_attr})

TODO: ...

In [ ]:
from thinc.api import Model, chain, docs2arrays, list2padded, with_array, chain, Embed, Padded, remap_ids
from spacy.attrs import LOWER

def WordEmbed(width, vocab) -> Model[List[Doc], Padded]:
    return chain(
        docs2arrays([LOWER]),
        list2padded(),
        with_array( # Padded -> Padded
            chain(remap_ids(vocab), Embed(nO=width, nV=len(vocab)))
        )
    )

The `docs2arrays` layer uses spaCy's [`Doc.to_array`](https://spacy.io/api/doc#to_array) method, giving you an array
with the features you asked for. In this case, we'll use the lowercased form of the word. We then sort the batch by length and pad it, returning the results in Thinc's `Padded` dataclass. At this point, the values in our batch are 64-bit hashes, as that's what spaCy's `Token.lower` attribute returns.

### Creating the `TextClassifier`

With our `WordEmbed` layer complete, we can put in some other reasonable choices for our text classifier, and create a model instance.


The `PyTorchLSTM` layer wraps PyTorch's LSTM implementation, taking care of the input and output transformations to work on Thinc's `Padded` type. We then use the `concatenate` combinator to build a feature representation using both max pooling and mean pooling. Pooling operations lose a lot of information, so using two together can result in slightly higher accuracy.

The `concatenate` combinator is an example of the type of relationship that you'd express in terms of instances: in PyTorch you'd write something like `Y = torch.concat(X.mean(axis=0), X.max(axis=0))`. This approach works fine, but the data variables (`X` and `Y`) can make it more difficult to see the relationships – and it's the relationships, the network topology, that you are usually most interested when you're reviewing the network. Once you get used to it, writing the network this way is also very easy.

In [ ]:
import spacy
from thinc.api import PyTorchLSTM, concatenate, MaxPool, MeanPool, chain, residual, ReLu, Softmax

nlp = spacy.blank("en")
vocab = {nlp.vocab.strings["hello"]: 0, nlp.vocab.strings["world"]: 1}
width = 128
n_class = 4

embed = WordEmbed(width, vocab)
encode = PyTorchLSTM(nO=width, nI=width, dropout=0.2, bi=True)
reduce = concatenate(MaxPool(), MeanPool())
predict = chain(
    residual(ReLu(nO=width*2, dropout=0.2, normalize=True)),
    Softmax(n_class)
)

model = TextClassifier(embed, encode, reduce, predict)

After creating the model, we then call its `initialize` method with a small batch of data, which gives layers a chance to guess missing dimensions and allocate parameters.

In [ ]:
inputs = [nlp.make_doc("hello world"), nlp.make_doc("example 2")]
labels = model.ops.asarray([[1.0, 0.0], [0.0, 1.0]], dtype="f")
model.initialize(X=inputs, Y=labels)

### Incorporating character features

Let's make the example more complex, to see how different requirements are handled. What if we wanted to incorporate character features into the embedding layer? We still want to embed the word IDs, but we _also_ want to run some sort of character-sensitive model, and combine the two representations before passing the result forward into the rest of the network.

There are lots of ways to do character-based features, but for this example, we'll encode the strings to UTF-8 byte sequences, embed the bytes, encode the sequences using an LSTM, and use the final hidden state as the character-based vector. We'll then sum the character-based vectors with the embedded word IDs.

For this new model, we'll have to do some refactoring of our `TextClassifier` and `WordEmbed` components. We want the two embedding layers to take the same input and output types, so that we can combine them easily. The most convenient signature would be `Model[List[str], Array2d]`: a flat list of strings for the whole batch as input, and a single array for the batch as output.

This requires a few changes to the `WordEmbed` layer we defined earlier. We'll remove the `list2padded` step, and instead of `docs2strings`, we'll define our own little lower-casing operation and apply the `strings2arrays` layer.

In [ ]:
from typing import List
from thinc.api import Model, chain, strings2arrays, with_array, chain, Embed, remap_ids
from thinc.types import Array2d

def WordEmbed(width, vocab) -> Model[List[str], Array2d]:
    return chain(
        lower_case(),
        strings2arrays(),
        with_array(
            chain(remap_ids(vocab), Embed(nO=width, nV=len(vocab)))
        )
    )

def lower_case() -> Model[List[str], List[str]]:
    def forward(Model[List[str], List[str]], strings: List[str], is_train: bool):
        return [string.lower() for string in strings], lambda dX: []
    
    return Model("lower_case", forward)

The `lower_case` transformation is not differentiable, but we still need to pass a callback for the backward pass, which should return the same type as the input. The `CharacterEmbed` layer also needs a little non-differentiable transformation, to encode the strings into UTF-8 and convert the bytes into arrays. Once we have that, we can chain the pieces together, much as we did for the document model, but with a different reduction strategy: instead of using reducing to the mean and max, we'll reduce by just taking the last vector of each sequence.

In [ ]:
def CharacterEmbed(width, depth) -> Model[List[str], Array2d]:
    return chain(
        strings2utf8(),
        list2padded(),
        with_array(Embed(width, 256)),
        PyTorchLSTM(width, width, depth=depth),
        reduce_nth(-1)
    )

def strings2utf8() -> Model[List[str]], List[Array2d]]:
    def forward(model: Model[List[str], List[Array2d], words: List[str], is_train: bool):                  
        utf8_arrays: List[str]] = []
        for word in words:
            chars = word.encode("utf8")
            ints = list(map(int.from_bytes, chars))
            arr = model.ops.asarray(ints, dtype="i").reshape((-1, 1))
            utf8_arrays.append(arr)
        return utf8_arrays, lambda dX: []
                
    return Model("strings2utf8", forward)

To wire everything together, we need to flatten the nested lists on the way into the embedding layer, and split the output array into lists of the correct length. The `with_flatten` function provides the necessary transformation. With all the pieces in place, we can finish updating our `TextClassifier` function, and pass in our combined embedding model, with one last finishing touch: the addition of a caching operation, `uniqued`, around the embedding layer.

In [ ]:
from typing import List
from thinc.api import Model, Padded, chain, with_flatten, list2padded, uniqued, PyTorchBiLSTM, concatenate, MaxPool, MeanPool, chain, residual, ReLu, Softmax
from thinc.types import Array2d
from spacy.tokens import Doc

def TextClassifier(
    embed: Model[List[str], Array2d],
    encode: Model[Padded, Padded],
    reduce: Model[Padded, Array2d],
    predict: Model[Array2d, Array2d]
) -> Model[List[Doc], Array2d]:
    return chain(
        docs2strings(),      # List[Doc] -> List[List[str]]
        with_flatten(embed), # List[List[str]] -> List[Array2d]
        list2padded(),
        encode,
        reduce,
        predict
    )

# TODO: What's word_embed? What's char_embed?

model = TextClassifer(
    uniqued(add(word_embed, char_embed)),
    PyTorchBiLSTM(width, width, dropout=0.0),
    concatenate(MaxPool(), MeanPool()),
    chain(
        residual(ReLu(nO=width*2, dropout=0.2, normalize=True)),
        Softmax(n_class)
    )
)

---

## Training the model

TODO: ...